In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [13]:
df=pd.read_csv("/content/WA_Fn-UseC_-HR-Employee-Attrition.csv")
df

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(hue=df.Attrition,x=df.Age)
plt.figure(figsize=(30,100))

In [ ]:
df.groupby(['Age'], as_index=False)['Attrition'].sum().sort_values(by='Attrition', ascending=False)

In [ ]:
ax=sns.countplot(x=df['BusinessTravel'],hue=df.Attrition,data=df)
for i in ax.containers:
  ax.bar_label(i)

In [ ]:
df.groupby(['BusinessTravel'],as_index=False)['Attrition'].sum().sort_values(by='Attrition',ascending=False)

In [ ]:
ax=sns.countplot(x=df['Department'],hue=df.Attrition,data=df)
for i in ax.containers:
  ax.bar_label(i)

In [ ]:
df.groupby(['Department'],as_index=False)['Attrition'].sum().sort_values(by='Attrition',ascending=False)

In [ ]:
ax=sns.countplot(x=df['EducationField'],hue=df.Attrition,data=df)
plt.xticks(rotation='vertical')
for i in ax.containers:
  ax.bar_label(i)

In [ ]:
df.groupby(['EducationField'],as_index=False)['Attrition'].sum().sort_values(by='Attrition',ascending=False)

In [ ]:
sns.boxplot(df['YearsAtCompany'])

df.groupby(['RelationshipSatisfaction'],as_index=False)['Attrition'].sum().sort_values(by='Attrition',ascending=False)

df.drop(columns=["EmployeeCount", "StandardHours"], inplace=True)


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

df['Attrition']= label_encoder.fit_transform(df['Attrition'])

df['Attrition'].unique()

cat_cols = df.select_dtypes(exclude=np.number).columns
num_cols = df.select_dtypes(include=np.number).columns

for i in cat_cols:
    print(f"\n====================> {i} \n")
    print(df[i].value_counts())

for i in num_cols:
    print(f"\n====================> {i} \n")
    print(df[i].value_counts())


In [ ]:
encoded_cat_cols = pd.get_dummies(df[cat_cols], drop_first=True)

encoded_cat_cols.info()

final_model = pd.concat([df[num_cols], encoded_cat_cols], axis=1)

final_model

cor=final_model.corr()

cor

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(cor,annot=True,linewidths=.5, fmt=".2f", cmap="YlGnBu")
plt.show()

final_model.drop(columns=["YearsAtCompany","YearsInCurrentRole", "YearsWithCurrManager", "Department_Sales", "PerformanceRating",
                         "JobLevel", "MonthlyIncome", "Department_Research & Development", "TotalWorkingYears", "BusinessTravel_Travel_Rarely",
                         "MaritalStatus_Married","MaritalStatus_Single"], inplace=True)

cor = final_model.corr()


In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(cor,annot=True,linewidths=.5, fmt=".2f", cmap="YlGnBu")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
x = final_model.drop(columns="Attrition")
y = final_model.Attrition
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=10)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model=GaussianNB()

model

In [ ]:
model.fit(x_train, y_train)
train_pred = model.predict(x_train)

In [ ]:
metrics.confusion_matrix(y_train, train_pred)

In [ ]:
accuracy_percentage_train = (metrics.accuracy_score(y_train, train_pred)) * 100
accuracy_percentage_train

In [ ]:
test_pred = model.predict(x_test)

In [ ]:
metrics.confusion_matrix(y_test, test_pred)

In [ ]:
accuracy_percentage_test = (metrics.accuracy_score(y_test, test_pred)) * 100
accuracy_percentage_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
dtree = DecisionTreeClassifier(class_weight='balanced')
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3, 4],
    'random_state': [0, 42]
}

In [ ]:
grid_search = GridSearchCV(dtree, param_grid, cv=5)
grid_search.fit(x_train, y_train)

In [ ]:
print(grid_search.best_params_) 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_leaf=3, min_samples_split=2, class_weight='balanced')
dtree.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = dtree.predict(x_test)
print("Accuracy Score :", round(accuracy_score(y_test, y_pred)*100 ,2), "%")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print('F-1 Score : ',(f1_score(y_test, y_pred, average='micro')))
print('Precision Score : ',(precision_score(y_test, y_pred, average='micro')))
print('Recall Score : ',(recall_score(y_test, y_pred, average='micro')))


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,5))
sns.heatmap(data=cm,linewidths=.5, annot=True,  cmap = 'Blues')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score for Decision Tree: {0}'.format(dtree.score(x_test, y_test))
plt.title(all_sample_title, size = 15)

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
lg=LogisticRegression()

In [ ]:
for i in range(0,100):
  x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=i)
  lg.fit(x_train,y_train)
  lgpred=lg.predict(x_test)
  print(f"at random_state{i},the testing accuracy score is : = {accuracy_score(y_test,lgpred)}")
  print(f"at random_state{i},the training accuracy is : = {lg.score(x_train,y_train)}")
